# The Nedelec space

Let's get acquainted with the Nedelec space for treating boundary value problems set in H(curl).

In [1]:
import netgen.gui
import ngsolve as ng

In [ ]:
from netgen.csg import CSGeometry, OrthoBrick, Cylinder, unit_cube, Pnt

from ngsolve import x, y, z

from ngsolve import Draw    
from ngsolve.internal import viewoptions, visoptions

## The lowest order Nedelec space

In [ ]:
mesh = ng.Mesh(unit_cube.GenerateMesh(maxh=0.5))

The lowest order Nedelec space can be generated in NGSolve by this:

In [ ]:
N = ng.HCurl(mesh, order=0, type1=True)

Note that the dimension of the space equals the number of mesh edges. 

In [ ]:
N.ndof

In [ ]:
len(mesh.edges)

## Nedelec shape functions

The Nedelec space has one shape function for each edge. 

In [ ]:
shape = ng.GridFunction(N, name='shape')
shape.vec[:] = 0
shape.vec[4] = 1

Draw(shape, mesh, 'shape', draw_surf=True)    

Note the tangential continuity of shape functions.  Pay particular attention to the tangential components along each edge of a tetrahedron. 

Visualization notes:
- In Netgen GUI, check the box `Draw Surface Vectors` in the `Visual` menu to see the vectors plotted. 
- In Netgen GUI,  you can go `Viewing Options` menu and check box `Show Edge-numbers` to see the edge numbering of the mesh when the `Mesh` tab is set for viewing the mesh.

## The interpolant

The Nedelec interpolant generates tangentially continuous approximants. To illustrate this point, we shall now make a mesh composed of two subdomains `lft` and `rgt`. Then we will interpolate a discontinuous vector field into the Nedelec space. 

In [ ]:
box1 = OrthoBrick(Pnt(-1, 0, 0), Pnt(0, 1, 1)).mat('lft')
box2 = OrthoBrick(Pnt(0, 0, 0), Pnt(1, 1, 1)).mat('rgt')

geo = CSGeometry()
geo.Add(box1)
geo.Add(box2)
mesh = ng.Mesh(geo.GenerateMesh(maxh=0.5))

It's easy to create a discontinuous vector field by defining it piecewise on each subdomain. Below, we pick a function whose tangential component is continuous   across the subdomain interface (contained in the $x=0$ plane).

In [ ]:
v = mesh.MaterialCF({'lft': (0,   -z, y),
                     'rgt': (-y, x-z, y)})

In [ ]:
viewoptions.clipping.enable=1
viewoptions.clipping.nx = 0 
viewoptions.clipping.ny = 0 
viewoptions.clipping.nz = -1
visoptions.clipsolution = 'vec'
Draw(v, mesh, 'v', draw_surf=False)

Note that the various viewing options can be listed using python's `dir`, e.g., `dir(viewoptions.clipping)`. You can also use the GUI to tune the viewing parameters.

In [ ]:
N = ng.HCurl(mesh, order=0, type1=True)
vh = ng.GridFunction(N)
vh.Set(v)  

In [ ]:
ng.sqrt(ng.Integrate((vh-v)**2, mesh))  

## Exercise: The 2D Nedelec space

In class exercise: Open a new Jupyter notebook. Make a 2D mesh of the unit square and examine the shape functions of the 2D Nedelec space. See if you can find a relationship with the shape functions of the 2D Raviart-Thomas space. 